In [1]:
import pandas as pd
import numpy as np
from pyltv import Model
from dbm import DBM
import plotly
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

# save local javascript for render plotly visualizations
from IPython.display import Javascript
Javascript(
"""require.config({
 paths: { 
     plotly: 'https://cdn.plot.ly/plotly-latest.min'
 }
});"""
)

from IPython.display import Javascript
from plotly.offline import get_plotlyjs
Javascript(get_plotlyjs())

ModuleNotFoundError: No module named 'pyltv'

In [ ]:
params = [
    'cumulative_ltv_per_original',
    'cumulative_dcf_ltv_per_original',
    'cumulative_cm$_per_original', 
    'cumulative_revenue_per_original',
    'cumulative_origination_per_original',
    'cumulative_opex_per_original',
    'cumulative_opex_coc_per_original',
    'cumulative_opex_cpl_per_original'
         ]

In [ ]:
dbm = DBM(user='kenny.liao')

In [4]:
ke_data = pd.read_csv('../data/ltv_report/ke_5-12.csv')
ph_data = pd.read_csv('../data/ltv_report/ph_5-12.csv')
mx_data = pd.read_csv('../data/ltv_report/mx_5-12.csv')

In [5]:
# generate models
ke = Model(ke_data, market='ke')
ph = Model(ph_data, market='ph')
mx = Model(mx_data, market='mx')

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17



In [6]:
# forecast data
ke.forecast = ke.forecast_data(ke.data, min_months=5, n_months=50)
ph.forecast = ph.forecast_data(ph.data, min_months=5, n_months=50)
mx.forecast = mx.forecast_data(mx.data, min_months=5, n_months=50)

In [7]:
# ke report
ke_ftb = ke.forecast[ke.forecast.index==1][['cohort', 'Count Borrowers']].set_index('cohort')
ke_ltv = ke.forecast[ke.forecast.index==50][['cohort'] + params].set_index('cohort')

ke_report = pd.concat([ke_ftb, ke_ltv], axis=1)

# ph report
ph_ftb = ph.forecast[ph.forecast.index==1][['cohort', 'Count Borrowers']].set_index('cohort')
ph_ltv = ph.forecast[ph.forecast.index==50][['cohort'] + params].set_index('cohort')

ph_report = pd.concat([ph_ftb, ph_ltv], axis=1)

# mx report
mx_ftb = mx.forecast[mx.forecast.index==1][['cohort', 'Count Borrowers']].set_index('cohort')
mx_ltv = mx.forecast[mx.forecast.index==50][['cohort'] + params].set_index('cohort')

mx_report = pd.concat([mx_ftb, mx_ltv], axis=1)

## LTV Report

### 18 mo LTVs

In [8]:
ke_ltv18 = round(ke.forecast[ke.forecast.index==18]['cumulative_ltv_per_original'].mean(), 2)
ph_ltv18 = round(ph.forecast[ph.forecast.index==18]['cumulative_ltv_per_original'].mean(), 2)
mx_ltv18 = round(mx.forecast[mx.forecast.index==18]['cumulative_ltv_per_original'].mean(), 2)
print(f'KE: ${ke_ltv18}')
print(f'PH: ${ph_ltv18}')
print(f'MX: ${mx_ltv18}')

KE: $15.74
PH: $30.01
MX: $36.15


### 50 mo LTVs

In [9]:
ke_ltv50 = round(ke.forecast[ke.forecast.index==50]['cumulative_ltv_per_original'].mean(), 2)
ph_ltv50 = round(ph.forecast[ph.forecast.index==50]['cumulative_ltv_per_original'].mean(), 2)
mx_ltv50 = round(mx.forecast[mx.forecast.index==50]['cumulative_ltv_per_original'].mean(), 2)
print(f'KE: ${ke_ltv50}')
print(f'PH: ${ph_ltv50}')
print(f'MX: ${mx_ltv50}')

KE: $40.53
PH: $76.15
MX: $67.85


### 50 mo Summary

In [10]:
ke_50 = round(ke.forecast[ke.forecast.index==50][params].mean(), 2)
ph_50 = round(ph.forecast[ph.forecast.index==50][params].mean(), 2)
mx_50 = round(mx.forecast[mx.forecast.index==50][params].mean(), 2)

summary = pd.concat([ke_50, ph_50, mx_50], axis=1)
summary.columns = ['KE', 'PH', 'MX']
summary

,KE,PH,MX
cumulative_ltv_per_original,40.53,76.15,67.85
cumulative_dcf_ltv_per_original,31.38,59.20,55.14
cumulative_cm$_per_original,59.02,95.48,90.02
cumulative_revenue_per_original,123.71,161.81,193.92
cumulative_origination_per_original,810.36,913.68,958.24
cumulative_opex_per_original,18.49,19.34,22.17
cumulative_opex_coc_per_original,8.78,9.90,10.38
cumulative_opex_cpl_per_original,9.71,9.44,11.79


## Plotting

In [46]:
ke.backtest, ke.backtest_report = ke.backtest_data(ke.data, min_months=5, hold_months=3, fcast_months=50)
mx.backtest, mx.backtest_report = mx.backtest_data(mx.data, min_months=5, hold_months=3, fcast_months=50)
ph.backtest, ph.backtest_report = ph.backtest_data(ph.data, min_months=5, hold_months=3, fcast_months=50)

Backtesting 3 months.
10 cohorts will be backtested.
Backtesting 3 months.
10 cohorts will be backtested.
Backtesting 3 months.
10 cohorts will be backtested.


In [59]:
ke.plot_cohorts('Count Borrowers', 'backtest')

In [18]:
(9188600/2547)/20

180.38084020416176